In [1]:
import matplotlib.pyplot as plt
import random
import torch
import pandas as pd
from data_utils import ucmerced_prepare_data
from train_utils import convnode_task

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/biasvariancelabs/aitlas-arena/refs/heads/main/splits/ucmerced_train.csv", header=None, names=['image_path', 'label'])
df[:3]

,image_path,label
0,river/river57.tif,river
1,intersection/intersection78.tif,intersection
2,sparseresidential/sparseresidential86.tif,sparseresidential


In [6]:
# Get unique classes
unique_classes = sorted(df['label'].unique())

# Create mappings
class_to_idx = {cls: idx for idx, cls in enumerate(unique_classes)}
idx_to_class = {idx: cls for cls, idx in class_to_idx.items()}

# Display the mappings
print("Class to Index Mapping:", class_to_idx)
print("Index to Class Mapping:", idx_to_class)

Class to Index Mapping: {'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
Index to Class Mapping: {0: 'agricultural', 1: 'airplane', 2: 'baseballdiamond', 3: 'beach', 4: 'buildings', 5: 'chaparral', 6: 'denseresidential', 7: 'forest', 8: 'freeway', 9: 'golfcourse', 10: 'harbor', 11: 'intersection', 12: 'mediumresidential', 13: 'mobilehomepark', 14: 'overpass', 15: 'parkinglot', 16: 'river', 17: 'runway', 18: 'sparseresidential', 19: 'storagetanks', 20: 'tenniscourt'}


In [2]:
def visualize_predictions(model, dataloader, num_samples=25):
    """
    Visualize ground truth vs. predicted labels for random samples from the dataloader.
    
    Args:
        model (torch.nn.Module): Trained PyTorch model.
        dataloader (DataLoader): Dataloader for the dataset.
        num_samples (int): Number of random examples to visualize.
    """
    model.eval()  # Set model to evaluation mode
    all_images, all_preds, all_labels = [], [], []

    # ImageNet normalization stats
    mean = torch.tensor([0.4914, 0.4822, 0.4465])
    std = torch.tensor([0.2023, 0.1994, 0.2010])

    with torch.no_grad():
        for batch in dataloader:
            images, labels = batch
            logits = model(images)
            preds = torch.argmax(logits, dim=1)

            all_images.extend(images.cpu())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Randomly select samples
    indices = random.sample(range(len(all_images)), num_samples)
    selected_images = [all_images[i] for i in indices]
    selected_preds = [all_preds[i] for i in indices]
    selected_labels = [all_labels[i] for i in indices]

    # Create a grid of images
    fig, axes = plt.subplots(5, 5, figsize=(15, 15))
    axes = axes.flatten()
    for img, pred, label, ax in zip(selected_images, selected_preds, selected_labels, axes):
        # De-normalize the image
        img = img.permute(1, 2, 0)  # Convert from CHW to HWC
        img = img * std + mean  # De-normalize (undo normalization)
        img = img * 255.0  # Scale back to 0-255
        img = img.numpy().astype('uint8')  # Convert to uint8 for visualization

        # Plot the image
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(f"GT: {label}\nPred: {pred}", color="green" if label == pred else "red")

    plt.tight_layout()
    plt.show()

In [2]:
train_loader, val_loader, labels = ucmerced_prepare_data.prepare_data()

ALL LABELS
['agricultural', 'airplane', 'baseballdiamond', 'beach', 'buildings', 'chaparral', 'denseresidential', 'forest', 'freeway', 'golfcourse', 'harbor', 'intersection', 'mediumresidential', 'mobilehomepark', 'overpass', 'parkinglot', 'river', 'runway', 'sparseresidential', 'storagetanks', 'tenniscourt']


In [ ]:
model = convnode_task.UCMERCDNeuralODE(num_classes=21)

In [13]:
model = convnode_task.UCMERCDNeuralODE.load_from_checkpoint("D:\\Projects\\neuralODE_RSclassification\\logs\\convode\\version_0\\checkpoints\\epoch=9-step=530.ckpt", num_classes=21)
model.to('cpu')
model.eval()

UCMERCDNeuralODE(
  (train_accuracy): MulticlassAccuracy()
  (val_accuracy): MulticlassAccuracy()
  (val_precision): MulticlassPrecision()
  (val_recall): MulticlassRecall()
  (val_f1): MulticlassF1Score()
  (val_mAP): MulticlassAveragePrecision()
  (val_math_corr_coeff): MulticlassMatthewsCorrCoef()
  (val_cohen_kappa): MulticlassCohenKappa()
  (confusion_matrix): MulticlassConfusionMatrix()
  (model): ConvODENet(
    (odeblock): ODEBlock(
      (odefunc): ConvODEFunc(
        (conv1): Conv2dTime(5, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv2): Conv2dTime(65, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2dTime(65, 4, kernel_size=(1, 1), stride=(1, 1))
        (non_linearity): ReLU(inplace=True)
      )
    )
    (linear_layer): Linear(in_features=200704, out_features=21, bias=True)
  )
)

In [18]:
input = torch.rand(100,3,224, 224)
with torch.no_grad():
    output = model(input)
print(output.shape)
print(output.argmax(dim=1))
print(output.argmax(dim=1).shape)

torch.Size([100, 21])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])
torch.Size([100])
